In [ ]:
import cv2
import numpy as np
import time
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.eval()

def calculate_speed(distance, time_interval):
    return distance / time_interval

cap = cv2.VideoCapture('/content/US_65.mp4')

vehicle_speeds = {}

fps = 30
distance_per_pixel = 0.05
time_interval = 1 / fps

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for result in results:
        for detection in result.boxes:
            x1, y1, x2, y2, conf, cls = detection.xyxy[0]
            if conf > 0.5:
                vehicle_type = model.names[int(cls)]
                center_x = (x1 + x2) / 2
                center_y = (y1 + y2) / 2

                distance = distance_per_pixel * 5
                speed = calculate_speed(distance, time_interval)

                if vehicle_type not in vehicle_speeds:
                    vehicle_speeds[vehicle_type] = []
                vehicle_speeds[vehicle_type].append(speed)

                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                cv2.putText(frame, f"{vehicle_type}: {speed:.2f} m/s", (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.imshow('Vehicle Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

average_speeds = {vtype: np.mean(speeds) for vtype, speeds in vehicle_speeds.items()}

print("Average Speeds by Vehicle Type:")
for vtype, avg_speed in average_speeds.items():
    print(f"{vtype}: {avg_speed:.2f} m/s")

cap.release()
cv2.destroyAllWindows()